<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC185.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [AtCoder Beginner Contest 185](https://atcoder.jp/contests/abc185)

## [A - ABC Preparation](https://atcoder.jp/contests/abc185/tasks/abc185_a)

In [ ]:
A1, A2, A3, A4 = map(int, input().split())
print(min(A1, A2, A3, A4))

## [B - Smartphone Addiction](https://atcoder.jp/contests/abc185/tasks/abc185_b)

In [ ]:
N, M, T = map(int, input().split())

now = N
time = 0
for _ in range(M):
    A, B = map(int, input().split())
    dec = A - time
    if now - dec <= 0:
        print("No")
        exit()
    now -= dec
    now = min(now+(B-A), N)
    time = B
else:
    dec = T - time
    if now - dec <= 0:
        print("No")
    else:
        print("Yes")

10 2 20
9 11
13 16
No


## [C - Duodecim Ferra](https://atcoder.jp/contests/abc185/tasks/abc185_c)

In [ ]:
L = int(input())
from math import factorial
print(factorial(L-1) // factorial(11) // factorial(L - 1 - 11))

200
366461620334848584


### Scipyでもできる

In [ ]:
from scipy.special import comb
L = int(input())
print(comb(L-1, 11, exact=True))

## [D - Stamp](https://atcoder.jp/contests/abc185/tasks/abc185_d)

In [ ]:
N, M = map(int, input().split())
A = list(map(int, input().split()))
A.sort()

5 2
1 3


In [ ]:
A = [0] + A + [N+1]
k = 10**10
for i in range(1, len(A)):
    if A[i] - A[i-1] > 1:
        k = min(k, A[i] - A[i-1] - 1)

In [ ]:
cnt = 0
for i in range(1, len(A)):
    if A[i] - A[i-1] > 1:
        cnt += -int(-(A[i] - A[i-1] - 1)//k)
print(cnt if k < 10**10 else 0)

1


## [E - Sequence Matchiing](https://atcoder.jp/contests/abc185/tasks/abc185_e)

まずは最長共通部分列のDPを復習する

In [ ]:
s = str(input())
t = str(input())

abracadabra
avadakedavra


In [ ]:
DP = [[0] * (len(t) + 1) for _ in range(len(s) + 1)]

In [ ]:
for i in range(1, len(s) + 1):
    for j in range(1, len(t) + 1):
        if s[i-1] == t[j-1]:
            DP[i][j] = DP[i-1][j-1] + 1
        else:
            DP[i][j] = max(DP[i][j], DP[i-1][j], DP[i][j-1])

この問題

In [ ]:
N, M = map(int, input().split())
A = list(map(int, input().split()))
B = list(map(int, input().split()))

4 3
1 2 1 3
1 3 1


In [ ]:
INF = 10**10
DP = [[INF] * (M+1) for _ in range(N+1)]

In [ ]:
for i in range(N+1):
    DP[i][0] = i
for j in range(M+1):
    DP[0][j] = j

In [ ]:
for i in range(1, N+1):
    for j in range(1, M+1):
        if A[i-1] == B[j-1]:
            y = 0
        else:
            y = 1

        DP[i][j] = min(DP[i][j], DP[i][j-1] + 1, DP[i-1][j] + 1, DP[i-1][j-1] + y)

In [ ]:
print(DP[-1][-1])

[[0, 1, 2, 3], [1, 0, 1, 2], [2, 1, 1, 2], [2, 2, 2, 1], [2, 3, 2, 2]]

## [F - Range Xor Query](https://atcoder.jp/contests/abc185/tasks/abc185_f)

In [ ]:
class SegmentTree:
    """ 1点更新・区間集約 Segment Tree. 非再起・1-index・非可換

    Attributes:
        tree (List[Union[int, float]]): セグメントツリーを表す配列
        segfunc (Callable): 集約を行う関数 (モノイド上の演算)
        ide_ele (float): 集約の関数における単位元　（モノイド上の単位元）
        update (int, float): 元の配列の更新 & セグメントツリーの更新
        query (int, int): 区間集約値の取得
    """
    def __init__(self, n: int, ide_ele: float, seg_func: callable):
        """
        Args:
            n (int): 元の配列のサイズ
            ide_ele (float): モノイドにおける単位元
            segfunc (callable): 集約を行う関数 （モノイド上の演算）
        """
        self.segfunc = seg_func
        self.ide_ele = ide_ele
        self.num = 1 << (n - 1).bit_length()
        self.tree = [self.ide_ele] * (2 * self.num)
    
    def update(self, i: int, x: float) -> None:
        """元の配列のi番目を x に変更（上書き）する. それに伴ってセグメントツリーも更新する. +=のようなupdateではない.

        Args:
            i (int): 元の配列のi番目の要素. 元の配列の0-indexの
            x (float): 変更後の値        
        """
        i += self.num
        self.tree[i] ^= x
        while i > 1:
            i >>= 1
            self.tree[i] = self.segfunc(self.tree[i << 1], self.tree[(i << 1) + 1])
        
    def query(self, l: int, r: int):
        """元の配列の閉区間[l, r]のsegfuncにおける集約値を取得する

        Args:
            l (int): 元の配列の集約を行う区間の左端のindex. 0-index
            r (int): 元の配列の集約を行う区間の右端のindex. 0-index
        """
        # [l, r) として葉から計算していく
        l += self.num
        r += self.num + 1

        l_res = self.ide_ele
        r_res = self.ide_ele
        
        while l < r:
            if l & 1:  # lが奇数だったらそのノードを集約する
                l_res = self.segfunc(l_res, self.tree[l])
                l += 1  # 右に進む
            if r & 1:  # rが奇数だったら, rは開区間なので その1つ左ノードを集約する
                r_res = self.segfunc(self.tree[r - 1], r_res)

            l >>= 1  # 上に進む（２で割って切り捨て）
            r >>= 1
        
        return self.segfunc(l_res, r_res)

In [ ]:
N, Q = map(int, input().split())
A = list(map(int, input().split()))
seg = SegmentTree(N, 0, lambda x, y: x^y)

3 4
1 2 3


In [ ]:
# 初期配列の設定
for i, x in enumerate(A):
    seg.update(i, x)

In [ ]:
# クエリ処理
for _ in range(Q):
    t, x, y = map(int, input().split())
    if t == 1:  # 更新
        seg.update(x-1, y)
    else:
        print(seg.query(x-1, y-1))

2 1 3
0
2 2 3
1
1 2 3
2 2 3
2
